In [17]:
import numpy as np
from sklearn.metrics import ndcg_score

In [41]:
y = 'US'

In [46]:
predictions = ['ES',  'GB', 'US','FR' ]

In [47]:
sum([(2**(predictions[i] == y) - 1)/(np.log2((i+1) + 1)) for i in range(len(predictions))])

0.5

array([1, 0, 0, 0])

In [66]:
rel_i  = np.where(np.asarray(list(map(lambda y_hat: y_hat == y, predictions))), 1, 0)

In [76]:
top = 2**rel_i - 1
top

array([0, 0, 1, 0])

In [77]:
bottom = np.log2(np.arange(1, len(rel_i) + 1, 1) + 1)
bottom

array([1.        , 1.5849625 , 2.        , 2.32192809])

In [78]:
answer = top / bottom

In [79]:
np.sum(answer)

0.5

In [168]:
def dcg(y, preds):
    rel_i = np.where(np.asarray(list(map(lambda y_hat: y_hat == y, preds))), 1, 0)
    
    return np.sum(
        (2**rel_i - 1) /
        np.log2(np.arange(1, len(rel_i) + 1) + 1)
    )

In [179]:
def ndcg(y, preds):
    return (
        dcg(y, preds) /
        dcg(y, [y])
    )

In [180]:
dcg('FR', ['US', 'FR'])

0.6309297535714575

In [199]:
ndcg('GB', ['FR', 'US', 'GB', 'MEX', 'POL'])

0.5

In [203]:
ndcg_score([[1]], [[0, 0, 1, 0, 0]], ignore_ties=True, k=5)

ValueError: Only ('multilabel-indicator', 'continuous-multioutput', 'multiclass-multioutput') formats are supported. Got binary instead

In [204]:
""" Reference from https://gist.github.com/bwhite/3726239
"""

import numpy as np

def dcg_at_k(r, k, method=0):
    """Score is discounted cumulative gain (dcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> dcg_at_k(r, 1)
    3.0
    >>> dcg_at_k(r, 1, method=1)
    3.0
    >>> dcg_at_k(r, 2)
    5.0
    >>> dcg_at_k(r, 2, method=1)
    4.2618595071429155
    >>> dcg_at_k(r, 10)
    9.6051177391888114
    >>> dcg_at_k(r, 11)
    9.6051177391888114
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Discounted cumulative gain
    """
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, k, method=0):
    """Score is normalized discounted cumulative gain (ndcg)
    Relevance is positive real values.  Can use binary
    as the previous methods.
    Example from
    http://www.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf
    >>> r = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    >>> ndcg_at_k(r, 1)
    1.0
    >>> r = [2, 1, 2, 0]
    >>> ndcg_at_k(r, 4)
    0.9203032077642922
    >>> ndcg_at_k(r, 4, method=1)
    0.96519546960144276
    >>> ndcg_at_k([0], 1)
    0.0
    >>> ndcg_at_k([1], 2)
    1.0
    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]
    Returns:
        Normalized discounted cumulative gain
    """
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max


print(ndcg_at_k([0],5, method=1))
print(ndcg_at_k([1],5, method=1))
print(ndcg_at_k([1,0],5, method=1))
print(ndcg_at_k([0,1],5, method=1))
print(ndcg_at_k([0,1,1],5, method=1))
print(ndcg_at_k([0,1,1,1],5, method=1))

0.0
1.0
1.0
0.6309297535714575
0.6934264036172708
0.7328286204777911


In [206]:
ndcg_at_k([0, 0, 1, 0, 0], 5, method=1)

0.5